In [1]:
from mindspore import context
context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import DynamicUpdater
from mindsponge.function import VelocityGenerator

## 单位设置

这里我们提到的全局单位设置，主要是指输入输出的长度单位和能量单位，在MindSponge的计算过程中会自动根据默认单位进行计算，再将返回的结果转换成用户定义的单位。常用的长度单位是nm和A，一般在PDB文件里面是用A为单位。能量单位比较常用的是kJ/mol和kcal/mol。

In [3]:
set_global_units('nm', 'kj/mol')

## 蛋白质案例

我们提供了一些简单的蛋白质案例以供测试，一般用pdb(Protein Data Bank)格式来存储蛋白质文件。下述案例就是一个不含H原子的pdb文件（该构象来自于MEGAProtein的预测结果）。

![](../../docs/case2-1.png)

在使用MindSponge构建Protein对象时，会自动为其在相对合理的位置补充H原子（如下图所示）。而这一过程之后，最好对其进行一次能量最小化的操作，以确保得到的构象中不会存在太多的冲突，比如两个原子距离太近等。

![](../../docs/case2-3.png)

如果使用常用的蛋白质可视化算法NewCartoon来展示的话，可以更清晰的看到这个构象中所存在的$\alpha$螺旋和$\beta$折叠等区域：

![](../../docs/case2-2.png)

In [4]:
pdb_name = 'case2.pdb'
system = Protein(pdb=pdb_name)

1 H-Adding task complete.


In [5]:
energy = ForceField(system, 'AMBER.FF14SB')

In [6]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)

In [7]:
md = Sponge(system, energy, min_opt)

In [8]:
run_info = RunInfo(10)
md.run(500, callbacks=[run_info])

Step: 0, E_pot: 141277.78, 
Step: 10, E_pot: 96736.516, 
Step: 20, E_pot: 75656.49, 
Step: 30, E_pot: 62071.246, 
Step: 40, E_pot: 52376.703, 
Step: 50, E_pot: 45027.336, 
Step: 60, E_pot: 39223.355, 
Step: 70, E_pot: 34505.492, 
Step: 80, E_pot: 30588.4, 
Step: 90, E_pot: 27282.95, 
Step: 100, E_pot: 24456.766, 
Step: 110, E_pot: 22013.084, 
Step: 120, E_pot: 19879.006, 
Step: 130, E_pot: 17998.32, 
Step: 140, E_pot: 16327.078, 
Step: 150, E_pot: 14830.46, 
Step: 160, E_pot: 13480.788, 
Step: 170, E_pot: 12255.744, 
Step: 180, E_pot: 11137.308, 
Step: 190, E_pot: 10110.747, 
Step: 200, E_pot: 9163.998, 
Step: 210, E_pot: 8287.033, 
Step: 220, E_pot: 7471.5547, 
Step: 230, E_pot: 6710.5693, 
Step: 240, E_pot: 5998.164, 
Step: 250, E_pot: 5329.3027, 
Step: 260, E_pot: 4699.6895, 
Step: 270, E_pot: 4105.588, 
Step: 280, E_pot: 3543.8252, 
Step: 290, E_pot: 3011.5625, 
Step: 300, E_pot: 2506.3613, 
Step: 310, E_pot: 2026.0889, 
Step: 320, E_pot: 1568.7969, 
Step: 330, E_pot: 1132.8047, 
S

In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)

In [10]:
opt = DynamicUpdater(system,
                     integrator=VelocityVerlet(system),
                     thermostat=Langevin(system, 300),
                     time_step=1e-3,
                     velocity=velocity)

In [11]:
md = Sponge(system, energy, min_opt)

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_b06.h5md', save_freq=10, write_velocity=True, write_force=True)

In [13]:
md.change_optimizer(opt)
md.run(2000, callbacks=[run_info, cb_h5md])

Step: 0, E_pot: -4076.3857, E_kin: 9336.439, E_tot: 5260.0537, Temperature: 296.12753
Step: 10, E_pot: -3602.2153, E_kin: 7402.628, E_tot: 3800.4126, Temperature: 234.79207
Step: 20, E_pot: -4049.0723, E_kin: 9462.248, E_tot: 5413.176, Temperature: 300.11786
Step: 30, E_pot: -3221.604, E_kin: 8893.889, E_tot: 5672.2847, Temperature: 282.09097
Step: 40, E_pot: -4569.623, E_kin: 10438.651, E_tot: 5869.0283, Temperature: 331.08682
Step: 50, E_pot: -4390.322, E_kin: 9904.172, E_tot: 5513.85, Temperature: 314.13452
Step: 60, E_pot: -4648.2305, E_kin: 9788.482, E_tot: 5140.252, Temperature: 310.46515
Step: 70, E_pot: -4359.8057, E_kin: 9773.725, E_tot: 5413.919, Temperature: 309.99707
Step: 80, E_pot: -4080.7705, E_kin: 9694.955, E_tot: 5614.1846, Temperature: 307.49872
Step: 90, E_pot: -4831.9717, E_kin: 10237.94, E_tot: 5405.9688, Temperature: 324.72076
Step: 100, E_pot: -4624.5195, E_kin: 9960.655, E_tot: 5336.1357, Temperature: 315.92603
Step: 110, E_pot: -4604.515, E_kin: 9770.462, E_to